Week 7 Notebook: Optimizing Other Objectives
===============================================================

This week, we will look at optimizing multiple objectives simultaneously. In particular, we will look at pivoting with adversarial neural networks {cite}`Louppe:2016ylz,ganin2014unsupervised,Sirunyan:2019nfw`.

In [1]:
import keras
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import uproot

Using TensorFlow backend.


In [2]:
# 48 track-level features
features = ['track_pt',
            'track_ptrel',
            'trackBTag_Eta',
            'trackBTag_DeltaR',
            'trackBTag_EtaRel',
            'trackBTag_JetDistVal',
            'trackBTag_Momentum',
            'trackBTag_PPar',
            'trackBTag_PParRatio',
            'trackBTag_PtRatio',
            'trackBTag_PtRel',
            'trackBTag_Sip2dSig',
            'trackBTag_Sip2dVal',
            'trackBTag_Sip3dSig',
            'trackBTag_Sip3dVal',
            'track_VTX_ass',
            'track_charge',
            'track_deltaR',
            'track_detadeta',
            'track_dlambdadz',
            'track_dlambdadz',
            'track_dphidphi',
            'track_dphidxy',
            'track_dptdpt',
            'track_drminsv',
            'track_drsubjet1',
            'track_drsubjet2',
            'track_dxy',
            'track_dxydxy',
            'track_dxydz',
            'track_dxysig',
            'track_dz',
            'track_dzdz',        
            'track_dzsig',
            'track_erel',
            'track_etarel',
            'track_fromPV',
            'track_isChargedHad',
            'track_isEl',
            'track_isMu',
            'track_lostInnerHits',
            'track_mass',
            'track_normchi2',            
            'track_phirel',
            'track_pt',
            'track_ptrel',
            'track_puppiw',
            'track_quality']

# spectators to define mass/pT window
spectators = ['fj_sdmass',
              'fj_pt']

# 2 labels: QCD or Hbb (we'll reduce the following labels)
labels =  ['label_QCD_b',
           'label_QCD_bb',
           'label_QCD_c', 
           'label_QCD_cc', 
           'label_QCD_others',
           'sample_isQCD',
           'label_H_bb']

nfeatures = len(features)
nspectators = len(spectators)
nlabels = 2

# we're going to zero-pad up to 60 tracks
ntracks = 60

In [3]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_files, features, labels, spectators, batch_size=1024, n_dim=60, 
                 remove_mass_pt_window=False, remove_unlabeled=True, return_spectators=False,
                 max_entry = 20000):
        'Initialization'
        self.batch_size = batch_size
        self.labels = labels
        self.list_files = list_files
        self.features = features
        self.spectators = spectators
        self.return_spectators = return_spectators
        self.n_dim = n_dim
        self.n_channels = len(features)
        self.remove_mass_pt_window = remove_mass_pt_window
        self.remove_unlabeled = remove_unlabeled
        self.global_IDs = []
        self.local_IDs = []
        self.file_mapping = []
        self.max_entry = max_entry
        self.open_files = [None]*len(self.list_files)
        running_total = 0
        for i, file_name in enumerate(self.list_files):
            root_file = uproot.open(file_name)
            self.open_files.append(root_file)
            tree = root_file['deepntuplizer/tree']
            tree_length = min(len(tree),max_entry)
            self.global_IDs.append(np.arange(running_total,running_total+tree_length))
            self.local_IDs.append(np.arange(tree_length))
            self.file_mapping.append(np.repeat(i,tree_length))
            running_total += tree_length
            root_file.close()
        self.global_IDs = np.concatenate(self.global_IDs)
        self.local_IDs = np.concatenate(self.local_IDs)
        self.file_mapping = np.concatenate(self.file_mapping)
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.global_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        files = self.file_mapping[index*self.batch_size:(index+1)*self.batch_size]
        
        unique_files = np.unique(files)
        starts = np.array([min(indexes[files==i]) for i in unique_files])
        stops = np.array([max(indexes[files==i]) for i in unique_files])

        # Check if files needed open (if not open them)
        # Also if file is not needed, close it
        for ifile, file_name in enumerate(self.list_files):
            if ifile in unique_files:
                if self.open_files[ifile] is None: 
                    self.open_files[ifile] = uproot.open(file_name)
            else:
                if self.open_files[ifile] is not None: 
                    self.open_files[ifile].close()
                    self.open_files[ifile] = None
            
        # Generate data
        return self.__data_generation(unique_files, starts, stops)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = self.local_IDs

    def __data_generation(self, unique_files, starts, stops):
        'Generates data containing batch_size samples' 
        # X : (n_samples, n_dim, n_channels)
        # y : (n_samples, 2)
        Xs = []
        ys = []
        zs = []
        
        # Generate data
        for ifile, start, stop in zip(unique_files, starts, stops):
            if self.return_spectators:
                X, y, z = self.__get_features_labels(ifile, start, stop)
                zs.append(z)
            else:
                X, y = self.__get_features_labels(ifile, start, stop)
            Xs.append(X)
            ys.append(y)
            
        # Stack data if going over multiple files
        if len(unique_files)>1:
            X = np.concatenate(Xs,axis=0)
            y = np.concatenate(ys,axis=0)
            if self.return_spectators:
                z = np.concatenate(zs,axis=0)
            
        if self.return_spectators:
            return X, [y, z]
        
        return X, y
                         
    def __get_features_labels(self, ifile, entrystart, entrystop):
        'Loads data from one file'
        
        # Double check that file is open
        if self.open_files[ifile] is None:
            root_file = uproot.open(self.list_file[ifile])
        else:
            root_file = self.open_files[ifile]
            
        tree = root_file['deepntuplizer/tree']
        
        feature_array = tree.arrays(branches=self.features, 
                                    entrystart=entrystart,
                                    entrystop=entrystop,
                                    namedecode='utf-8')

        label_array_all = tree.arrays(branches=self.labels, 
                                      entrystart=entrystart,
                                      entrystop=entrystop,
                                      namedecode='utf-8')

        X = np.stack([feature_array[feat].pad(self.n_dim, clip=True).fillna(0).regular() for feat in features],axis=2)
        n_samples = X.shape[0]
    
        y = np.zeros((n_samples,2))
        y[:,0] = label_array_all['sample_isQCD'] * (label_array_all['label_QCD_b'] + \
                                                    label_array_all['label_QCD_bb'] + \
                                                    label_array_all['label_QCD_c'] + \
                                                    label_array_all['label_QCD_cc'] + \
                                                    label_array_all['label_QCD_others'])
        y[:,1] = label_array_all['label_H_bb']

        
        if self.remove_mass_pt_window:
            # remove data outside of mass/pT range
            spec_array = tree.arrays(branches=self.spectators, 
                                     entrystart=entrystart,
                                     entrystop=entrystop,
                                     namedecode='utf-8')
            
            z = np.stack([spec_array[spec] for spec in self.spectators],axis=1)
            X = X[(z[:,0] > 40) & (z[:,0] < 200) & (z[:,1] > 300) & (z[:,1] < 2000)]
            y = y[(z[:,0] > 40) & (z[:,0] < 200) & (z[:,1] > 300) & (z[:,1] < 2000)]
            z = z[(z[:,0] > 40) & (z[:,0] < 200) & (z[:,1] > 300) & (z[:,1] < 2000)]
                        
        if self.remove_unlabeled:
            # remove unlabeled data
            X = X[np.sum(y,axis=1)==1]
            if self.return_spectators:
                z = z[np.sum(y,axis=1)==1]
            y = y[np.sum(y,axis=1)==1]
            
        if self.return_spectators:
            return X, [y, z]
        
        return X, y

## Define Gradient Reversal Layer
Here, we define a gradient reversal layer

In [13]:
@tf.custom_gradient
def grad_reverse(x):
    y = tf.identity(x)
    def custom_grad(dy):
        return -dy
    return y, custom_grad

class GradReverse(keras.layers.Layer):
    def __init__(self, name='GradReverse'):
        super().__init__(name=name)

    def call(self, x):
        return grad_reverse(x)

## Define Combined Model with Adversarial Neural Network

In [16]:
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Conv1D, Flatten, Lambda, Concatenate
import keras.backend as K

# define Deep Sets model with Conv1D Keras layer
inputs = Input(shape=(ntracks,nfeatures,), name = 'input')  
x = BatchNormalization(name='bn_1')(inputs)
x = Conv1D(64, 1, strides=1, padding='same', name = 'conv1d_1', activation='relu')(x)
x = Conv1D(32, 1, strides=1, padding='same', name = 'conv1d_2', activation='relu')(x)
x = Conv1D(32, 1, strides=1, padding='same', name = 'conv1d_3', activation='relu')(x)
# sum over tracks
x = Lambda(lambda x: K.mean(x, axis=-2), name = 'lambda_1', input_shape=(ntracks,32))(x)
x = Dense(100, name = 'dense_1', activation='relu')(x)
output = Dense(nlabels, name = 'output', activation='softmax')(x)
 
# start adversarial part
x = GradReverse(name='reverse_1')(output)
x = Dense(100, name='dense_2', activation ='relu')(x)
x = Dense(100, name= 'dense_3',activation='relu')(x)
output_reg = Dense(2, activation='linear', name='mass_reg')(x)
outputs = Concatenate(name='concat_2')([output_reg, output])
                                                            

keras_model_adv = Model(inputs=inputs, outputs=outputs)
keras_model_adv.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(keras_model_adv.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 60, 48)       0                                            
__________________________________________________________________________________________________
bn_1 (BatchNormalization)       (None, 60, 48)       192         input[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 60, 64)       3136        bn_1[0][0]                       
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 60, 32)       2080        conv1d_1[0][0]                   
____________________________________________________________________________________________